In [29]:
!pip install praw
!pip install psaw
!pip install requests
!pip install sentence_transformers
!pip install detoxify
!pip install language-tool-python

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
import praw
import pandas as pd
import datetime
import time
import re
from detoxify import Detoxify

In [32]:
#Usefull functions
#def get_date(submission):
#	time = submission.created
#	return datetime.datetime.fromtimestamp(time).strftime("%d/%m/%Y")

In [33]:
#Check similatiries between two strings
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2')

def similar(a, b):
  #return re.findall("\d+\.\d+", str(util.dot_score(a, b)))[0]
  first = model.encode(a)
  second = model.encode(b)
  return util.dot_score(first, second)

In [34]:
#My credentials
#reddit = praw.Reddit(client_id ='1zriW2v1Y6SJqC8gOa5bZg',
#                     client_secret ='3dHrMCJ75qcXUJav18N9OG3HXt2u1w',
#                     user_agent ='garra')
#                     check_for_async=False)
# to verify whether the instance is read-only instance or not
#print(reddit.read_only)

In [35]:
#Select the subreddit
#subreddit = reddit.subreddit('facts')
# display the subreddit name
#print(subreddit.display_name)
 
# display the subreddit title
# print(subreddit.title)      
 
# display the subreddit description
#print(subreddit.description)

In [36]:
#RESET THE DATAGRAME; BE CAREFULL!!!
#today = date.today()
#d1 = today.strftime("%d/%m/%Y")
#data = {"Fact": ["The opposite of Microsoft Office is Macrohard Onfire"], "Holiday":["General"]}
#post = pd.DataFrame(data)
#post.to_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Project/Wrappers/Facts.xlsx")

In [37]:
#Read the data
post = pd.read_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Project/Wrappers/Facts.xlsx", index_col=0)

In [38]:

#Take the posts and insert it

#for submission in subreddit.top("month", limit=50):
#    escape = False
#    for fact in post.RedditFact:
#        if similar(fact, submission.title) > 0.9:
#            escape = True
#    if escape == False:
#        new_row = {"Date": get_date(submission), "RedditFact": submission.title}
#        post = post.append(new_row, ignore_index=True)
        # print(submission.title)

In [39]:
#Take the posts and insert it
#for submission in subreddit.top("all", limit=50):
#    if post["RedditFact"].str.contains(submission.title).any() == False:
#        new_row = {"Date": get_date(submission), "RedditFact": submission.title}
#        post = post.append(new_row, ignore_index=True)
        # print(submission.title)

In [40]:
####Post de los últimos 2 meses, se puede ajustar para el periodo de tiempo querido, limite de 1000 post
#current_timestamp = time.time()
#### 60 seconds * 60 minutes * 24 hours * 60 days = 2 months
#two_months_timestamp = current_timestamp - (60 * 60 * 24 * 60)
#query = 'timestamp:{}..{}'.format(current_timestamp, two_months_timestamp)
#for i in reddit.subreddit('facts').search(query = query, sort='top'):
#  print(i.title)

In [41]:
from psaw import PushshiftAPI
import datetime as dt

import language_tool_python
tool = language_tool_python.LanguageTool('en-US')

d = {'fact': ["example"]}
df = pd.DataFrame(data=d)

#YEAR - MONTH - DAY
#The dataframe starts at 2015 1 1 and goes in interval of 6 months
start_epoch=int(dt.datetime(2018, 6, 1).timestamp())
print(start_epoch)
end_epoch=int(dt.datetime(2018, 12, 31).timestamp())
api = PushshiftAPI()
listexam = list(api.search_submissions(after=start_epoch,
                            before=end_epoch,
                            subreddit='facts',
                            sort_type="score",
                            sort="desc",
                            filter=['title'],
                            limit=100))
print(listexam)
for i in listexam:
  toxi = Detoxify('original').predict(str(i.title))
  #If it contains any type of toxicity
  if any(v > 0.7 for v in iter(toxi.values())):
    continue
  #Shorts fact tend to be titles and not facts
  if (len(i.title) < 60) or (len(i.title) > 180):
    continue
  #If it starts with a number
  if ((i.title[0] == "0")or(i.title[0] == "1")or(i.title[0] == "2")or(i.title[0] == "3")or(i.title[0] == "4")or(i.title[0] == "5")or(i.title[0] == "6")or(i.title[0] == "7")or(i.title[0] == "8")or(i.title[0] == "9")):
    continue
  #If it is a question
  if (str(i.title[-1]) == "?"):
    continue
  #If it is grammaticly incorrect
  if len(tool.check(i.title)) > 1:
    continue
  #Check specific words
  if ("fact" in i.title) or ("Fact" in i.title) or ("Top" in i.title) or ("tact" in i.title):
    continue
  print(i.title)
  
  #Check the topic
  topic = "General"
  christmas = ["christmas", "merry", "festive", "santa claus", "jolly", "reindeer", "carols", "xmas", "mistletoe", "nativity", "presents", "sledge", "holly", "snow", "snowball", "snowman", "carol", "december"]
  valentine = ["cupid", "heart", "love", "roses", "crush", "boyfriend", "date", "kiss", "marriage", "sweet", "soulmates", "valentine"]
  fouls = ["fools", "comical", "funny", "humor", "foolish", "gag", "hilarious", "joke", "prank", "spoof", "trick"]
  patrick = ["patrick", "alcohol", "beer", "party", "celtic", "coins", "four-leaf", "fortune", "gold", "luck", "irish", "ireland", "lucky"]

  if any(word in i.title.lower() for word in christmas):
    topic = "Christmas"
  elif any(word in i.title.lower() for word in valentine):
    topic = "Valentine"
  elif any(word in i.title.lower() for word in fouls):
    topic = "Fouls"
  elif any(word in i.title.lower() for word in patrick):
    topic = "Patrick"

  #Check if the fact is already in the dataframe
  escape = False
  for fact in post.Fact:
      if similar(fact, str(i.title)) > 0.9:
          escape = True
  if escape == False:
      new_row = {"Holiday": topic, "Fact": str(i.title)}
      post = post.append(new_row, ignore_index=True)
        # print(submission.title)

1527811200


/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


[submission(created_utc=1530368628, title='Escaping from jail in Germany', created=1530368628.0, d_={'created_utc': 1530368628, 'title': 'Escaping from jail in Germany', 'created': 1530368628.0}), submission(created_utc=1528435707, title='We Always Had Despacito 2', created=1528435707.0, d_={'created_utc': 1528435707, 'title': 'We Always Had Despacito 2', 'created': 1528435707.0}), submission(created_utc=1531836990, title='Cleopatra lived closer in time to the invention of the iPhone than to the construction of the Great Pyramids.', created=1531836990.0, d_={'created_utc': 1531836990, 'title': 'Cleopatra lived closer in time to the invention of the iPhone than to the construction of the Great Pyramids.', 'created': 1531836990.0}), submission(created_utc=1531459487, title='Fact: Chemotherapy is the most expensive Haircut', created=1531459487.0, d_={'created_utc': 1531459487, 'title': 'Fact: Chemotherapy is the most expensive Haircut', 'created': 1531459487.0}), submission(created_utc=15

In [42]:
post.to_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Project/Wrappers/Facts.xlsx")